In [41]:
## Check whether a customer will remain a customer of the bank or exit.

In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [43]:
## Load the dataset

data = pd.read_csv('./Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [44]:
## Preprocess the data
## Drop the irrelevent feature (1st 3 rows are not useful)

df = data.copy()
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis= 1)
df


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [45]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [46]:
## encode categorical variable

label_encoder_gender = LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [47]:
## Onehot encode 'Geography'

from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(df[['Geography']]).toarray()
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [48]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [49]:
geo_encoder_df = pd.DataFrame(geo_encoder, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [50]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [51]:
## Combine all the columns with the original data

data_1  = pd.concat([df.drop('Geography',axis=1), geo_encoder_df], axis =1)
data_1.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [52]:
## Save the encoders and scalars

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo, file)


In [53]:
## Divide the dataset into independent and dependent features

X = data_1.drop('Exited', axis= 1)
y = data_1['Exited']

## Split the data in training and testing set
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [54]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [55]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler, file)

In [56]:
# ANN Implementation

In [57]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime


In [58]:
(X_train.shape[1],)

(12,)

In [59]:
## Build our ANN Model

model = Sequential([
     Dense(64, activation= 'relu', input_shape= (X_train.shape[1],)),  ## HL1 Conected with input layer
     Dense(32, activation ='relu'),  ## HL2
     Dense(1, activation='sigmoid')  ## Output layer

])

In [60]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [61]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate = 0.01)
loss= tensorflow.keras.losses.BinaryCrossentropy()
loss
opt


In [62]:
## Compile the model
model.compile(optimizer=opt, loss='binary_crossentropy', metrics = ['accuracy'])

In [63]:
## Set up the Tensorboard

from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq = 1)

In [64]:
## Set up Early Stopping

early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights=True)

In [65]:
## Train the model

history = model.fit(
         X_train,y_train, validation_data = (X_test,y_test), epochs = 100,
         callbacks = [tensorflow_callback, early_stopping_callback]
    
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3944 - accuracy: 0.8385 - val_loss: 0.3550 - val_accuracy: 0.8570
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3551 - accuracy: 0.8565 - val_loss: 0.3667 - val_accuracy: 0.8515
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3478 - accuracy: 0.8577 - val_loss: 0.3560 - val_accuracy: 0.8505
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3429 - accuracy: 0.8619 - val_loss: 0.3554 - val_accuracy: 0.8525
Epoch 5/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3405 - accuracy: 0.8618 - val_loss: 0.3401 - val_accuracy: 0.8610
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3356 - accuracy: 0.8635 - val_loss: 0.3372 - val_accuracy: 0.8605
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3348 - accuracy: 0.8616 - val_loss: 0.3356 - val_accuracy: 0.8560

In [66]:
model.save('model.h5py')

INFO:tensorflow:Assets written to: model.h5py\assets


INFO:tensorflow:Assets written to: model.h5py\assets


In [67]:
!pip install tensorboard


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [68]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [69]:
## Load Tensorboard Extension using magic command (%)

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [70]:
import os
os.environ["TENSORBOARD_BINARY"] = "/path/to/tensorboard"  # Replace with the actual path from step 4
%load_ext tensorboard
%tensorboard --logdir logs/fit/20250327-080019

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Could not find '/path/to/tensorboard' (set by the
`TENSORBOARD_BINARY` environment variable). Please ensure that your
PATH contains an executable `tensorboard` program, or explicitly
specify the path to a TensorBoard binary by setting the
`TENSORBOARD_BINARY` environment variable.

In [71]:
import os
print(os.path.exists("logs/fit/20250327-040905"))

True


In [72]:
import os
print(os.path.exists("logs/fit/20250327-080019"))

True


In [73]:
!where tensorboard

INFO: Could not find files for the given pattern(s).


In [74]:
%tensorboard --logdir logs/fit/20250327-080019

ERROR: Could not find '/path/to/tensorboard' (set by the
`TENSORBOARD_BINARY` environment variable). Please ensure that your
PATH contains an executable `tensorboard` program, or explicitly
specify the path to a TensorBoard binary by setting the
`TENSORBOARD_BINARY` environment variable.